# Problem formulation

**_Sets_**

$\mathscr{N}$ = Set of nodes <br>
$\mathscr{A}$ = Set of links <br>
$\mathscr{K}$ = Set of O-D pairs <br>

**_Parameters_**

$\beta_{ij}$ = Parameter of BPR function, $\forall (i,j) \in \mathscr{A}$ <br>
$t^0_{ij}$ = Free-flow travel time, $\forall (i,j) \in \mathscr{A}$ <br>
$\alpha_{ij}$ = Power in BPR function, $\forall (i,j) \in \mathscr{A}$ <br>
$c_{ij}$ = Capacity of link, $\forall (i,j) \in \mathscr{A}$ <br>
$d_{ik}$ = Demand for O-D pair $k$ at node $i$, $\forall i,k \in \mathscr{N} \times \mathscr{K}$ <br>
$M$ = node incidence matrix $(M_{ia})$, $\forall i,a \in \mathscr{N} \times \mathscr{A}$ 


**_Variables_**

$f_{ijk}$ = Link flow for O-D pair $k$, $\forall (i,j), k \in \mathscr{A} \times \mathscr{K}$  <br>
$f_{ij}$ = Total link flow, $\forall (ij) \in \mathscr{A}$

From now on, we have: $\mathbf{a = (i,j)}$

**_Objective_**


**UE**: $\min z = \sum_a t^0_a[f_a + \frac{\beta_a}{\alpha_a + 1} \cdot f_a \cdot (\frac{f_a}{c_a})^{\alpha_a}]$ <br>
**SO**: $\min z = \sum_a t^0_a[1+\beta_a\cdot (\frac{f_a}{c_a})^{\alpha_a}] \cdot f_a$ 

**_Constraints_**

$f_a = \sum_k f_{ak}$, $\forall a \in \mathscr{A}$ -> Definitional constraints on link flow <br>
$M\cdot(f_{ak}) - (d_{ik}) = 0$, $ \forall k \in \mathscr{K}$ -> Flow balance constraint <br> 
$f_{ak} \geq 0$, $\forall k \in \mathscr{K}$ -> Non negativity constraints on link flow 


# Outline

# 1. Loading and manipulating the data
For *Anaheim_net*:
- We need a function that downloads the data and create a list for each of the parameters $\beta$, $t^0$, $\alpha$, $c$
- We need a function that creates an array **_A_** of tuples for the links using the first two columns. 
- We need a function that takes **_A_** and creates the (sparse) node incidence matrix **_M_**.

For *Anaheim_trips*:
- We need a function that creates the (sparse) vector **_dk_** using the above file. 

# 2. An abstract Pyomo model
Here, for this we need: 
- A definition of each of the model sets and parameters
- A definition of the model objective function
- A definition of the model constraint functions 

# 3. A function for solving the model
Here, we need a function that will solve take the model and solver options as input and return the solution

# 4. A function for results post-processing
Here, we need a function that will output the result to the desired format (if we think the Pyomo ouput is not convenient) and <br>
a function that will write the results to a specified file format

**NOTE**: 
- Having functions here would make it easier for us to build a class later on which will be helpful when we move to the AV case where we will do a lot of evaluations
- We should create each function in a separate .py file. Though we will probably use Jupyter for testing everything, having a .py file for each function helps make <br>
    the code modular and eases notebook manipulation

In [1]:
#Python libraries
from pyomo.environ import *
# Pyomo library for solving a problem
from pyomo.opt import SolverFactory
# Pyomo library for checking the solver statuse 
from pyomo.opt import SolverStatus, TerminationCondition
#panda is a useful data manipulation library
import numpy as np
import pandas as pd
from scipy import sparse
from sys import getsizeof
import time

In [2]:
#Homemade functions
from getParam import getParam
from getDemand import getDemand
from objRule import objRule
from flowBalRule import flowBalRule
from flowLinkRule import flowLinkRule
from costLinkRule import costLinkRule
from postPro import postPro

filenameNet = "Data/SiouxFalls/SiouxFalls_net.tntp"

param,A,M,N,L = getParam(filenameNet)

filenameTrip = "Data/SiouxFalls/SiouxFalls_trips.tntp"

d, OD_pair= getDemand(filenameTrip,L)

# Model definitions

In [3]:
model=ConcreteModel()

model.links=RangeSet(0,L-1,1) #Set of links
model.od=RangeSet(0,len(OD_pair)-1,1) #Set of OD pairs
model.nodes=RangeSet(0,N-1,1) #Set of Nodes
model.f=Var(model.links,within=NonNegativeReals) #set of link flows
model.fk=Var(model.links,model.od,within=NonNegativeReals) #Set link-OD flows
model.c=Var(model.links,within=NonNegativeReals) #Set of link costs


In [4]:
model.obj = Objective(expr=objRule(model))  #Objective function

In [5]:
model.flowBal =  ConstraintList()
t0 = time.time()
for i in model.nodes:
    for k in model.od:
        #model.flowBal.add(flowBalRule(model,d,M,i,k))
        model.flowBal.add(sum(j[1]*model.fk[j[0],k] for j in M[i])-d[i,k]==0.0)
        
t1 = time.time()
print("Elapsed time: "+str(t1-t0))

Elapsed time: 6.154674530029297


In [6]:
model.flowLink =  ConstraintList()
model.costLink = ConstraintList()
for a in model.links:
    model.flowLink.add(flowLinkRule(model,a))
    model.costLink.add(costLinkRule(model,param,a))
#model.flowLink = Constraint(model.links, rule=flowLinkRule)

In [7]:
opt = SolverFactory("ipopt")

#opt.options["mipgap"] = 0.05
#store the results 
results = opt.solve(model)
print(results)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 13400
  Number of variables: 42104
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.11.1\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 180.90435314178467
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [8]:
if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
    print("Optimality is Achevied")# Do something when the solution in optimal and feasible
elif (results.solver.termination_condition == TerminationCondition.infeasible):
    print("Model is infeasible")
else:
    # Something else is wrong
    print ("Solver Status: ",  results.solver.status)

Optimality is Achevied


In [9]:
instance=model

In [10]:
print("The objective function value is ", value(instance.obj))

The objective function value is  7194256.042342926


In [11]:
for v in instance.component_objects(Var, active=True):
    print ("Variable",v.name)
    varobject = getattr(instance, str(v))
    for index in varobject:
        print(varobject[index].name,index, varobject[index].value,)

Variable f
f[0] 0 7620.034016572735
f[1] 1 11239.63351878345
f[2] 2 7639.633518743418
f[3] 3 6620.034016526657
f[4] 4 11220.034016612792
f[5] 5 17501.113530466715
f[6] 6 14313.666600350429
f[7] 7 17474.033076117208
f[8] 8 18732.187520023752
f[9] 9 6080.494139867669
f[10] 10 18800.26716001136
f[11] 11 6994.497863757055
f[12] 12 16895.599441157607
f[13] 13 6639.63351869733
f[14] 14 7014.338112154931
f[15] 15 12556.6220952646
f[16] 16 13272.89240398696
f[17] 17 16324.35393343345
f[18] 18 12596.061845833145
f[19] 19 13224.353933256165
f[20] 20 7566.033696842069
f[21] 21 7967.9084500327
f[22] 22 16943.838832747435
f[23] 23 7535.640288003836
f[24] 24 21764.782483207302
f[25] 25 21882.62846595884
f[26] 26 17467.774406186367
f[27] 27 23361.19499877268
f[28] 28 10744.945867647057
f[29] 29 8320.005338885778
f[30] 30 6085.334045530469
f[31] 31 17387.458154132975
f[32] 32 7324.915056727184
f[33] 33 9443.00971034576
f[34] 34 14321.147552529264
f[35] 35 7323.879298153316
f[36] 36 15538.581662617547


fk[1,12] (1, 12) 299.9999999900604
fk[1,13] (1, 13) 499.9999999899765
fk[1,14] (1, 14) 0.0
fk[1,15] (1, 15) 399.9999999899088
fk[1,16] (1, 16) 0.0
fk[1,17] (1, 17) 299.99999998992996
fk[1,18] (1, 18) 0.0
fk[1,19] (1, 19) 99.99999998996877
fk[1,20] (1, 20) 399.9999999898706
fk[1,21] (1, 21) 299.99999999006957
fk[1,22] (1, 22) 99.99999999007792
fk[1,23] (1, 23) 0.0
fk[1,24] (1, 24) 99.99999999007602
fk[1,25] (1, 25) 199.99999999003245
fk[1,26] (1, 26) 99.99999998998658
fk[1,27] (1, 27) 0.0
fk[1,28] (1, 28) 0.0
fk[1,29] (1, 29) 0.0
fk[1,30] (1, 30) 199.99999998998055
fk[1,31] (1, 31) 599.9999999899762
fk[1,32] (1, 32) 199.99999999003003
fk[1,33] (1, 33) 99.99999999007491
fk[1,34] (1, 34) 299.99999999007474
fk[1,35] (1, 35) 99.99999999001899
fk[1,36] (1, 36) 99.99999998986509
fk[1,37] (1, 37) 0.0
fk[1,38] (1, 38) 199.99999998962323
fk[1,39] (1, 39) 0.0
fk[1,40] (1, 40) 99.99999998973715
fk[1,41] (1, 41) 0.0
fk[1,42] (1, 42) 0.0
fk[1,43] (1, 43) 99.99999998930477
fk[1,44] (1, 44) 0.0
fk[1,4

fk[2,127] (2, 127) 0.0
fk[2,128] (2, 128) 0.0
fk[2,129] (2, 129) 0.0
fk[2,130] (2, 130) 0.0
fk[2,131] (2, 131) 0.0
fk[2,132] (2, 132) 0.0
fk[2,133] (2, 133) 0.0
fk[2,134] (2, 134) 0.0
fk[2,135] (2, 135) 0.0
fk[2,136] (2, 136) 49.91906440737711
fk[2,137] (2, 137) 52.61209475386198
fk[2,138] (2, 138) 499.9999999891568
fk[2,139] (2, 139) 0.0
fk[2,140] (2, 140) 44.38196440159952
fk[2,141] (2, 141) 0.0
fk[2,142] (2, 142) 0.0
fk[2,143] (2, 143) 0.0
fk[2,144] (2, 144) 0.0
fk[2,145] (2, 145) 0.0
fk[2,146] (2, 146) 0.0
fk[2,147] (2, 147) 0.0
fk[2,148] (2, 148) 436.07208015100014
fk[2,149] (2, 149) 218.43593869780307
fk[2,150] (2, 150) 0.0
fk[2,151] (2, 151) 0.0
fk[2,152] (2, 152) 0.0
fk[2,153] (2, 153) 0.0
fk[2,154] (2, 154) 0.0
fk[2,155] (2, 155) 0.0
fk[2,156] (2, 156) 0.0
fk[2,157] (2, 157) 0.0
fk[2,158] (2, 158) 0.0
fk[2,159] (2, 159) 0.0
fk[2,160] (2, 160) 0.0
fk[2,161] (2, 161) 799.9999999893788
fk[2,162] (2, 162) 0.0
fk[2,163] (2, 163) 96.27823810225124
fk[2,164] (2, 164) 0.0
fk[2,165] (2

fk[3,240] (3, 240) 0.0
fk[3,241] (3, 241) 0.0
fk[3,242] (3, 242) 0.0
fk[3,243] (3, 243) 0.0
fk[3,244] (3, 244) 0.0
fk[3,245] (3, 245) 0.0
fk[3,246] (3, 246) 0.0
fk[3,247] (3, 247) 0.0
fk[3,248] (3, 248) 0.0
fk[3,249] (3, 249) 0.0
fk[3,250] (3, 250) 0.0
fk[3,251] (3, 251) 0.0
fk[3,252] (3, 252) 0.0
fk[3,253] (3, 253) 0.0
fk[3,254] (3, 254) 0.0
fk[3,255] (3, 255) 0.0
fk[3,256] (3, 256) 0.0
fk[3,257] (3, 257) 0.0
fk[3,258] (3, 258) 110.10125070396327
fk[3,259] (3, 259) 432.12690732435095
fk[3,260] (3, 260) 356.5146759534919
fk[3,261] (3, 261) 0.0
fk[3,262] (3, 262) 0.0
fk[3,263] (3, 263) 0.0
fk[3,264] (3, 264) 0.0
fk[3,265] (3, 265) 0.0
fk[3,266] (3, 266) 0.0
fk[3,267] (3, 267) 432.126906072026
fk[3,268] (3, 268) 0.0
fk[3,269] (3, 269) 95.63339568712843
fk[3,270] (3, 270) 0.0
fk[3,271] (3, 271) 0.0
fk[3,272] (3, 272) 0.0
fk[3,273] (3, 273) 0.0
fk[3,274] (3, 274) 0.0
fk[3,275] (3, 275) 0.0
fk[3,276] (3, 276) 0.0
fk[3,277] (3, 277) 0.0
fk[3,278] (3, 278) 0.0
fk[3,279] (3, 279) 0.0
fk[3,280]

fk[4,354] (4, 354) 0.0
fk[4,355] (4, 355) 0.0
fk[4,356] (4, 356) 0.0
fk[4,357] (4, 357) 0.0
fk[4,358] (4, 358) 0.0
fk[4,359] (4, 359) 0.0
fk[4,360] (4, 360) 0.0
fk[4,361] (4, 361) 0.0
fk[4,362] (4, 362) 0.0
fk[4,363] (4, 363) 0.0
fk[4,364] (4, 364) 0.0
fk[4,365] (4, 365) 0.0
fk[4,366] (4, 366) 0.0
fk[4,367] (4, 367) 0.0
fk[4,368] (4, 368) 399.9999999899088
fk[4,369] (4, 369) 199.9999999896232
fk[4,370] (4, 370) 0.0
fk[4,371] (4, 371) 0.0
fk[4,372] (4, 372) 0.0
fk[4,373] (4, 373) 0.0
fk[4,374] (4, 374) 0.0
fk[4,375] (4, 375) 0.0
fk[4,376] (4, 376) 0.0
fk[4,377] (4, 377) 0.0
fk[4,378] (4, 378) 0.0
fk[4,379] (4, 379) 0.0
fk[4,380] (4, 380) 0.0
fk[4,381] (4, 381) 0.0
fk[4,382] (4, 382) 0.0
fk[4,383] (4, 383) 0.0
fk[4,384] (4, 384) 0.0
fk[4,385] (4, 385) 0.0
fk[4,386] (4, 386) 0.0
fk[4,387] (4, 387) 0.0
fk[4,388] (4, 388) 0.0
fk[4,389] (4, 389) 0.0
fk[4,390] (4, 390) 0.0
fk[4,391] (4, 391) 0.0
fk[4,392] (4, 392) 0.0
fk[4,393] (4, 393) 0.0
fk[4,394] (4, 394) 0.0
fk[4,395] (4, 395) 0.0
fk[4,3

...

fk[72,75] (72, 75) 0.0
fk[72,76] (72, 76) 0.0
fk[72,77] (72, 77) 0.0
fk[72,78] (72, 78) 0.0
fk[72,79] (72, 79) 0.0
fk[72,80] (72, 80) 0.0
fk[72,81] (72, 81) 0.0
fk[72,82] (72, 82) 0.0
fk[72,83] (72, 83) 0.0
fk[72,84] (72, 84) 0.0
fk[72,85] (72, 85) 0.0
fk[72,86] (72, 86) 0.0
fk[72,87] (72, 87) 0.0
fk[72,88] (72, 88) 0.0
fk[72,89] (72, 89) 0.0
fk[72,90] (72, 90) 0.0
fk[72,91] (72, 91) 0.0
fk[72,92] (72, 92) 0.0
fk[72,93] (72, 93) 0.0
fk[72,94] (72, 94) 0.0
fk[72,95] (72, 95) 0.0
fk[72,96] (72, 96) 0.0
fk[72,97] (72, 97) 0.0
fk[72,98] (72, 98) 0.0
fk[72,99] (72, 99) 0.0
fk[72,100] (72, 100) 0.0
fk[72,101] (72, 101) 0.0
fk[72,102] (72, 102) 0.0
fk[72,103] (72, 103) 0.0
fk[72,104] (72, 104) 0.0
fk[72,105] (72, 105) 0.0
fk[72,106] (72, 106) 0.0
fk[72,107] (72, 107) 0.0
fk[72,108] (72, 108) 0.0
fk[72,109] (72, 109) 0.0
fk[72,110] (72, 110) 0.0
fk[72,111] (72, 111) 0.0
fk[72,112] (72, 112) 0.0
fk[72,113] (72, 113) 0.0
fk[72,114] (72, 114) 0.0
fk[72,115] (72, 115) 0.0
fk[72,116] (72, 116) 0.0


fk[72,511] (72, 511) 87.9546487530857
fk[72,512] (72, 512) 0.0
fk[72,513] (72, 513) 0.0
fk[72,514] (72, 514) 139.47509476589013
fk[72,515] (72, 515) 0.0
fk[72,516] (72, 516) 0.0
fk[72,517] (72, 517) 699.9999999898431
fk[72,518] (72, 518) 799.99999998988
fk[72,519] (72, 519) 0.0
fk[72,520] (72, 520) 0.0
fk[72,521] (72, 521) 0.0
fk[72,522] (72, 522) 0.0
fk[72,523] (72, 523) 0.0
fk[72,524] (72, 524) 0.0
fk[72,525] (72, 525) 0.0
fk[72,526] (72, 526) 694.1819451061323
fk[72,527] (72, 527) 0.0
fk[72,528] (72, 528) 699.9999999900359
fk[72,529] (72, 529) 0.0
fk[72,530] (72, 530) 0.0
fk[72,531] (72, 531) 0.0
fk[72,532] (72, 532) 0.0
fk[72,533] (72, 533) 0.0
fk[72,534] (72, 534) 0.0
fk[72,535] (72, 535) 0.0
fk[72,536] (72, 536) 0.0
fk[72,537] (72, 537) 0.0
fk[72,538] (72, 538) 0.0
fk[72,539] (72, 539) 0.0
fk[72,540] (72, 540) 0.0
fk[72,541] (72, 541) 0.0
fk[72,542] (72, 542) 0.0
fk[72,543] (72, 543) 0.0
fk[72,544] (72, 544) 0.0
fk[72,545] (72, 545) 0.0
fk[72,546] (72, 546) 0.0
fk[72,547] (72, 54

fk[73,400] (73, 400) 0.0
fk[73,401] (73, 401) 0.0
fk[73,402] (73, 402) 0.0
fk[73,403] (73, 403) 99.9999999870209
fk[73,404] (73, 404) 0.0
fk[73,405] (73, 405) 0.0
fk[73,406] (73, 406) 0.0
fk[73,407] (73, 407) 0.0
fk[73,408] (73, 408) 0.0
fk[73,409] (73, 409) 0.0
fk[73,410] (73, 410) 0.0
fk[73,411] (73, 411) 0.0
fk[73,412] (73, 412) 0.0
fk[73,413] (73, 413) 0.0
fk[73,414] (73, 414) 0.0
fk[73,415] (73, 415) 0.0
fk[73,416] (73, 416) 0.0
fk[73,417] (73, 417) 0.0
fk[73,418] (73, 418) 0.0
fk[73,419] (73, 419) 0.0
fk[73,420] (73, 420) 0.0
fk[73,421] (73, 421) 0.0
fk[73,422] (73, 422) 0.0
fk[73,423] (73, 423) 0.0
fk[73,424] (73, 424) 0.0
fk[73,425] (73, 425) 0.0
fk[73,426] (73, 426) 0.0
fk[73,427] (73, 427) 0.0
fk[73,428] (73, 428) 0.0
fk[73,429] (73, 429) 0.0
fk[73,430] (73, 430) 0.0
fk[73,431] (73, 431) 0.0
fk[73,432] (73, 432) 0.0
fk[73,433] (73, 433) 0.0
fk[73,434] (73, 434) 0.0
fk[73,435] (73, 435) 0.0
fk[73,436] (73, 436) 0.0
fk[73,437] (73, 437) 0.0
fk[73,438] (73, 438) 0.0
fk[73,439] (

fk[74,348] (74, 348) 0.0
fk[74,349] (74, 349) 0.0
fk[74,350] (74, 350) 0.0
fk[74,351] (74, 351) 0.0
fk[74,352] (74, 352) 0.0
fk[74,353] (74, 353) 0.0
fk[74,354] (74, 354) 0.0
fk[74,355] (74, 355) 0.0
fk[74,356] (74, 356) 0.0
fk[74,357] (74, 357) 0.0
fk[74,358] (74, 358) 0.0
fk[74,359] (74, 359) 0.0
fk[74,360] (74, 360) 0.0
fk[74,361] (74, 361) 0.0
fk[74,362] (74, 362) 0.0
fk[74,363] (74, 363) 0.0
fk[74,364] (74, 364) 0.0
fk[74,365] (74, 365) 0.0
fk[74,366] (74, 366) 0.0
fk[74,367] (74, 367) 0.0
fk[74,368] (74, 368) 0.0
fk[74,369] (74, 369) 0.0
fk[74,370] (74, 370) 0.0
fk[74,371] (74, 371) 0.0
fk[74,372] (74, 372) 0.0
fk[74,373] (74, 373) 0.0
fk[74,374] (74, 374) 0.0
fk[74,375] (74, 375) 0.0
fk[74,376] (74, 376) 0.0
fk[74,377] (74, 377) 0.0
fk[74,378] (74, 378) 0.0
fk[74,379] (74, 379) 0.0
fk[74,380] (74, 380) 0.0
fk[74,381] (74, 381) 0.0
fk[74,382] (74, 382) 0.0
fk[74,383] (74, 383) 0.0
fk[74,384] (74, 384) 0.0
fk[74,385] (74, 385) 0.0
fk[74,386] (74, 386) 0.0
fk[74,387] (74, 387) 0.0


fk[75,156] (75, 156) 0.0
fk[75,157] (75, 157) 0.0
fk[75,158] (75, 158) 0.0
fk[75,159] (75, 159) 0.0
fk[75,160] (75, 160) 0.0
fk[75,161] (75, 161) 0.0
fk[75,162] (75, 162) 0.0
fk[75,163] (75, 163) 0.0
fk[75,164] (75, 164) 0.0
fk[75,165] (75, 165) 0.0
fk[75,166] (75, 166) 0.0
fk[75,167] (75, 167) 0.0
fk[75,168] (75, 168) 0.0
fk[75,169] (75, 169) 0.0
fk[75,170] (75, 170) 0.0
fk[75,171] (75, 171) 0.0
fk[75,172] (75, 172) 0.0
fk[75,173] (75, 173) 0.0
fk[75,174] (75, 174) 0.0
fk[75,175] (75, 175) 0.0
fk[75,176] (75, 176) 0.0
fk[75,177] (75, 177) 0.0
fk[75,178] (75, 178) 0.0
fk[75,179] (75, 179) 0.0
fk[75,180] (75, 180) 0.0
fk[75,181] (75, 181) 0.0
fk[75,182] (75, 182) 0.0
fk[75,183] (75, 183) 0.0
fk[75,184] (75, 184) 0.0
fk[75,185] (75, 185) 0.0
fk[75,186] (75, 186) 0.0
fk[75,187] (75, 187) 0.0
fk[75,188] (75, 188) 0.0
fk[75,189] (75, 189) 0.0
fk[75,190] (75, 190) 0.0
fk[75,191] (75, 191) 0.0
fk[75,192] (75, 192) 0.0
fk[75,193] (75, 193) 0.0
fk[75,194] (75, 194) 0.0
fk[75,195] (75, 195) 0.0


In [12]:
varlist = []
indexlist = []

for v in instance.component_objects(Var, active=True):
    print ("Variable",v.name)
    varobject = getattr(instance, str(v))
    for index in varobject:
        varlist.append( varobject[index].value)
        indexlist.append(index)

Variable f
Variable fk
Variable c


In [13]:
result_series = pd.Series(varlist,index=indexlist)
result_series.to_csv('results_so.csv')